# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

### Business Understanding

 **The questions of interest for the Starbucks dataset are as follows:**

* Which demographic groups respond best to which offer type?
* Which demographic groups will make purchases even if they don't receive an offer?
* Which demographic groups spends the most amount?
* Build a machine learning model that predicts how much someone will spend based on demographics and offer type.
* Build a model that predicts whether or not someone will respond to an offer.



### Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('../data/dataset/portfolio.json', orient='records', lines=True)
profile = pd.read_json('../data/dataset/profile.json', orient='records', lines=True)
transcript = pd.read_json('../data/dataset/transcript.json', orient='records', lines=True)

In [2]:
portfolio.head()

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


In [3]:
# Check shape
portfolio.shape

(10, 6)

In [4]:
# Check for null values
portfolio.isna().sum()

channels      0
difficulty    0
duration      0
id            0
offer_type    0
reward        0
dtype: int64

In [5]:
# Unique offers
list(portfolio['offer_type'].unique())

[u'bogo', u'informational', u'discount']

In [6]:
# Min, Max and Median
portfolio.describe()

,difficulty,duration,reward
count,10.000000,10.000000,10.000000
mean,7.700000,6.500000,4.200000
std,5.831905,2.321398,3.583915
min,0.000000,3.000000,0.000000
25%,5.000000,5.000000,2.000000
50%,8.500000,7.000000,4.000000
75%,10.000000,7.000000,5.000000
max,20.000000,10.000000,10.000000


In [7]:
# Top offer type
portfolio['offer_type'].describe()

count       10
unique       3
top       bogo
freq         4
Name: offer_type, dtype: object

In [8]:
# Cleaning Channels

def channel_1(x):
    
    try:
        value = x[0]
        
        return value
    
    except:
       
        return float("NAN") 
    
    
def channel_2(x):
    
    try:
        value = x[1]
        
        return value
    
    except:
       
        return float("NAN")
        
def channel_3(x):
    
    try:
        value = x[2]
        
        return value
    
    except:
       
        return float("NAN") 
    
    
def channel_4(x):
    
    try:
        value = x[3]
        
        return value
    
    except:
       
        return float("NAN")     
    

portfolio['channel_1'] = portfolio['channels'].apply(channel_1)
portfolio['channel_2'] = portfolio['channels'].apply(channel_2)
portfolio['channel_3'] = portfolio['channels'].apply(channel_3)
portfolio['channel_4'] = portfolio['channels'].apply(channel_4)

In [9]:
portfolio

,channels,difficulty,duration,id,offer_type,reward,channel_1,channel_2,channel_3,channel_4
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,email,mobile,social,NaN
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,web,email,mobile,social
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0,web,email,mobile,NaN
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,web,email,mobile,NaN
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,web,email,NaN,NaN
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3,web,email,mobile,social
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2,web,email,mobile,social
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0,email,mobile,social,NaN
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,web,email,mobile,social
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2,web,email,mobile,NaN


In [10]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [11]:
# Check shape
profile.shape

(17000, 5)

In [12]:
# Check for null values
profile.isna().sum()

age                    0
became_member_on       0
gender              2175
id                     0
income              2175
dtype: int64

In [13]:
# Min, Max and Median
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


In [14]:
# Top gender
profile['gender'].describe()

count     14825
unique        3
top           M
freq       8484
Name: gender, dtype: object

In [15]:
# Unique id
len(list(profile['id'].unique()))

17000

In [16]:
# Unique gender
list(profile['gender'].unique())

[None, u'F', u'M', u'O']

In [17]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{u'offer id': u'9b98b8c7a33c4b65b9aebfe6a799e6...
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{u'offer id': u'0b1e1539f2cc45b7b9fa7c272da2e1...
2,offer received,e2127556f4f64592b11af22de27a7932,0,{u'offer id': u'2906b810c7d4411798c6938adc9daa...
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{u'offer id': u'fafdcd668e3743c1bb461111dcafc2...
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{u'offer id': u'4d5c57ea9a6940dd891ad53e9dbe8d...


In [18]:
# Check shape
transcript.shape

(306534, 4)

In [19]:
# Check for null values
transcript.isna().sum()

event     0
person    0
time      0
value     0
dtype: int64

In [20]:
# Min, Max and Median
transcript.describe()

,time
count,306534.000000
mean,366.382940
std,200.326314
min,0.000000
25%,186.000000
50%,408.000000
75%,528.000000
max,714.000000


In [21]:
# Top event
transcript['event'].describe()

count          306534
unique              4
top       transaction
freq           138953
Name: event, dtype: object

In [22]:
# Unique event
list(transcript['event'].unique())

[u'offer received', u'offer viewed', u'transaction', u'offer completed']

In [23]:
# Unique person
len(list(transcript['person'].unique()))

17000

In [24]:
# Cleaning value
def offer(x):
    try:
       
        value =  x['offer id']
        
        return value
        
    except:
        
        try: 
            
            value =  x['offer_id']
        
            return value
        
        except:
            
            return float("NAN")
  
    
def amount(x):
    try:
       
        value =  x['amount']
        
        return value
        
    except:
        
        return float("NAN")     

transcript['offer id'] = transcript['value'].apply(offer)
transcript['amount'] = transcript['value'].apply(amount)

In [25]:
transcript.head()

,event,person,time,value,offer id,amount
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{u'offer id': u'9b98b8c7a33c4b65b9aebfe6a799e6...,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{u'offer id': u'0b1e1539f2cc45b7b9fa7c272da2e1...,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,offer received,e2127556f4f64592b11af22de27a7932,0,{u'offer id': u'2906b810c7d4411798c6938adc9daa...,2906b810c7d4411798c6938adc9daaa5,NaN
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{u'offer id': u'fafdcd668e3743c1bb461111dcafc2...,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{u'offer id': u'4d5c57ea9a6940dd891ad53e9dbe8d...,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN


### Data Understanding

Now we have the question, we need to move the question into the data. Find the columns from the datasets that would answer these questions.

**The columns identified to answer the necessary questions are as below:**


* Which demographic groups respond best to which offer type?
 -  age, gender, became_member_on, income, offer type
 
* Which demographic groups will make purchases even if they don't receive an offer?
 -  age, offer id, gender, became_member_on, income, amount, person  

* Which demographic groups spends the most amount?
 -  age, offer id, gender, became_member_on, income, amount, person  
 
* Build a machine learning model that predicts how much someone will spend based on demographics and offer type.
 -   
 
* Build a model that predicts whether or not someone will respond to an offer.
 - 


### Data Preparation

In [26]:
portfolio.head()

,channels,difficulty,duration,id,offer_type,reward,channel_1,channel_2,channel_3,channel_4
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,email,mobile,social,NaN
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,web,email,mobile,social
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0,web,email,mobile,NaN
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,web,email,mobile,NaN
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,web,email,NaN,NaN


In [27]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [28]:
transcript.head()

,event,person,time,value,offer id,amount
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{u'offer id': u'9b98b8c7a33c4b65b9aebfe6a799e6...,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{u'offer id': u'0b1e1539f2cc45b7b9fa7c272da2e1...,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,offer received,e2127556f4f64592b11af22de27a7932,0,{u'offer id': u'2906b810c7d4411798c6938adc9daa...,2906b810c7d4411798c6938adc9daaa5,NaN
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{u'offer id': u'fafdcd668e3743c1bb461111dcafc2...,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{u'offer id': u'4d5c57ea9a6940dd891ad53e9dbe8d...,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN


In [29]:
def clean_data(profile, portfolio, transcript):
    
    # Rename column 'id' to 'person'
    profile = profile.rename(columns={'id': 'person'})
    
    # Rename column 'id' to 'offer id'
    portfolio = portfolio.rename(columns={'id': 'offer id'})
    
    # Merge dataframes proflie and transcript
    merged_df = profile.merge(transcript, how='right', on='person')
    
    # Drop Nan values in column 'Gender', 'Income'
    merged_df = merged_df.dropna(subset=['income'])
    
    # Drop column 'value'
    merged_df.drop(columns=['value'], inplace=True)
    
    # Create offer dataframe - offer_df
    offer_df = merged_df.dropna(subset=['offer id'])
    
    # Drop column 'amount' from offer_df dataframe
    offer_df.drop(columns=['amount'], inplace=True)
    
    # Merge dataframes portfolio and offer_df, map columns 'offer id' to 'offer type'
    offer_type_df = portfolio.merge(offer_df, how='right', on='offer id')
    
    # Create amount dataframe - amount_df
    amount_df = merged_df.dropna(subset=['amount'])
    
    # Drop column 'offer id'
    amount_df.drop(columns=['offer id'], inplace=True)
    
    return offer_type_df, amount_df  
    
    
offer_type_df, amount_df = clean_data(profile, portfolio, transcript)    

/home/jyothish/.local/lib/python2.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [30]:
# Which demographic groups respond best to which offer type
# Which demographic groups will make purchases even if they don't receive an offer
# Which demographic groups spends the most amount?
# Build a machine learning model that predicts how much someone will spend based on demographics and offer type.
# Build a model that predicts whether or not someone will respond to an offer

### Statistical Analysis

* Which demographic groups respond best to which offer type?

In [31]:
# Demographic groups and offer

In [32]:
# Demographic group Age

group_offer_age_df = offer_type_df.groupby(['age'])['offer_type'].describe() 

In [33]:
group_offer_age_df.head()

,count,unique,top,freq
age,,,,
18,668,3,bogo,290
19,1225,3,bogo,540
20,1213,3,bogo,572
21,1292,3,discount,540
22,1199,3,discount,525


In [34]:
# Offer types among age

group_offer_age_df.groupby(['top'])['freq'].sum()

top
bogo        43159
discount    21830
Name: freq, dtype: int64

In [35]:
# The demographic group 'age' responds best to 'bogo' offer type       

In [36]:
# Demographic group Gender

group_offer_gender_df = offer_type_df.groupby(['gender'])['offer_type'].describe()

In [37]:
group_offer_gender_df.head()

,count,unique,top,freq
gender,,,,
F,63719,3,bogo,27619
M,82896,3,bogo,35301
O,2190,3,discount,920


In [38]:
# Offer types among gender

group_offer_gender_df.groupby(['top'])['freq'].sum()

top
bogo        62920
discount      920
Name: freq, dtype: int64

In [39]:
# The demographic group gender responds best to bogo offer type     

In [40]:
# Demographic group became_member_on

group_offer_became_member_on_df = offer_type_df.groupby(['became_member_on'])['offer_type'].describe()

In [41]:
group_offer_became_member_on_df.head()

,count,unique,top,freq
became_member_on,,,,
20130729,15,2,discount,9
20130730,24,3,bogo,9
20130731,8,2,bogo,5
20130801,12,1,bogo,12
20130802,21,3,discount,12


In [42]:
# Offer types among became_member_on

group_offer_became_member_on_df.groupby(['top'])['freq'].sum()

top
bogo             39107
discount         32398
informational      270
Name: freq, dtype: int64

In [43]:
# The demographic group became_member_on responds best to bogo offer type     

In [44]:
# Demographic group income

group_offer_income_df = offer_type_df.groupby(['income'])['offer_type'].describe()

In [45]:
group_offer_income_df.head()

,count,unique,top,freq
income,,,,
30000.0,778,3,bogo,354
31000.0,2028,3,bogo,892
32000.0,2069,3,bogo,919
33000.0,1980,3,bogo,827
34000.0,1801,3,bogo,767


In [46]:
# Offer types among income

group_offer_income_df.groupby(['top'])['freq'].sum()

top
bogo        40037
discount    25455
Name: freq, dtype: int64

In [47]:
# The demographic group income responds best to bogo offer type     

* Which demographic groups will make purchases even if they don't receive an offer?

In [48]:
def match(offer_type_df, amount_df):

    persons_completed_offer = list(offer_type_df['person'][offer_type_df['event'] == 'offer completed'].unique())

    df_match = amount_df[amount_df['age'] == 144]

    for person in persons_completed_offer:

        df_match = pd.concat([df_match, amount_df[amount_df['person'].isin([person])]])
    
    
    return df_match


df_match = match(offer_type_df, amount_df)

In [49]:
# Find difference of dataframes

diff_df = amount_df[~amount_df.apply(tuple,1).isin(df_match.apply(tuple,1))]

In [50]:
# The demographic groups that will make purchases even if they don't receive an offer

diff_df.head()

,age,became_member_on,gender,person,income,event,time,amount
282,61,20180501,F,a448667f336b42c9a66fc5ffd5d73772,40000.0,transaction,66,1.31
283,61,20180501,F,a448667f336b42c9a66fc5ffd5d73772,40000.0,transaction,216,0.52
284,61,20180501,F,a448667f336b42c9a66fc5ffd5d73772,40000.0,transaction,222,10.07
286,61,20180501,F,a448667f336b42c9a66fc5ffd5d73772,40000.0,transaction,420,3.14
288,61,20180501,F,a448667f336b42c9a66fc5ffd5d73772,40000.0,transaction,492,2.79


* Which demographic groups spends the most amount?

In [51]:
# Make demographic groups with amount

In [52]:
amount_age_df = amount_df.groupby(['age'])['amount'].sum().to_frame()

In [53]:
# Top 10 age which spends the most amount

amount_age_df = amount_age_df.sort_values(by=['amount'], ascending=False)

amount_age_df.head(10)

,amount
age,
58,56201.03
53,50985.19
55,49928.62
57,48138.62
59,43947.55
54,43417.10
67,42483.99
51,42257.66
63,41243.36


In [54]:
amount_gender_df = amount_df.groupby(['gender'])['amount'].sum().to_frame()

In [55]:
# Top gender which spends the most amount

amount_gender_df = amount_gender_df.sort_values(by=['amount'], ascending=False)

amount_gender_df.head()

,amount
gender,
F,863695.00
M,844890.86
O,26356.54


In [56]:
amount_became_member_on_df = amount_df.groupby(['became_member_on'])['amount'].sum().to_frame()

In [57]:
# Top 10 became_member_on which spends the most amount

amount_became_member_on_df = amount_became_member_on_df.sort_values(by=['amount'], ascending=False)

amount_became_member_on_df.head(10)

,amount
became_member_on,
20171114,4950.71
20170815,4556.10
20170924,4038.07
20171007,3990.83
20171207,3826.20
20161111,3694.93
20170202,3681.06
20170807,3673.41
20171212,3492.65


In [58]:
amount_income_df = amount_df.groupby(['income'])['amount'].sum().to_frame()

In [59]:
# Top 10 income which spends the most amount

amount_income_df = amount_income_df.sort_values(by=['amount'], ascending=False)

amount_income_df.head(10)

,amount
income,
73000.0,41677.94
72000.0,39436.83
71000.0,37808.33
74000.0,35315.08
70000.0,33107.45
75000.0,32037.00
64000.0,31644.61
67000.0,27831.57
61000.0,27827.53


In [61]:
# Build a machine learning model that predicts how much someone will spend based on demographics and offer type.

In [62]:
# Build a model that predicts whether or not someone will respond to an offer.